# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: 
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [1]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [3]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [2]:
import os
import requests
import pandas as pd

In [5]:
SYMBOL = 'AAPL'
#USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
USE_ALPHA = False
os.environ['ALPHAVANTAGE_API_KEY'] = "APPL"
def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    print(js)

    keys = [k for k in js if 'Time Series' in k]
    if not keys:
        print('Error: "Time Series" key not found in API response. Full response:', js)
        df_api = pd.DataFrame(columns=['date','adj_close'])
    else:
        key = keys[0]
        df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
        df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()
    print(df_api)
v_api = validate(df_api, ['date','adj_close']); v_api


C:\Users\kavin\AppData\Local\Temp\ipykernel_2192\446969600.py:26: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()
[*********************100%***********************]  1 of 1 completed

Price        Date       Close        High         Low        Open    Volume
Ticker                   AAPL        AAPL        AAPL        AAPL      AAPL
0      2025-05-20  206.625504  208.233679  204.797576  207.434583  42496600
1      2025-05-21  201.860901  206.805286  200.482476  204.937411  59211800
2      2025-05-22  201.131729  202.520153  199.473607  200.482472  46742400
3      2025-05-23  195.048645  197.475883  193.240699  193.450453  78432900
4      2025-05-27  199.983047  200.512445  197.206185  198.075209  56288500
..            ...         ...         ...         ...         ...       ...
59     2025-08-14  232.779999  235.119995  230.850006  234.059998  51916300
60     2025-08-15  231.589996  234.279999  229.339996  234.000000  56038700
61     2025-08-18  230.889999  233.119995  230.110001  231.699997  37476200
62     2025-08-19  230.559998  232.869995  229.350006  231.279999  39320800
63     2025-08-20  226.449997  230.469894  225.889999  229.970001  22366967

[64 rows x 

{'missing': ['date', 'adj_close'], 'shape': (64, 6), 'na_total': 0}

In [6]:
import os
from pathlib import Path
from datetime import datetime
import pandas as pd

# Define base folder for saving
RAW = Path("raw")
RAW.mkdir(parents=True, exist_ok=True)   # create folder if missing

# Timestamp helper
def ts(fmt: str = "%Y%m%d_%H%M%S") -> str:
    """Return current timestamp as string."""
    return datetime.now().strftime(fmt)

def save_csv(df: pd.DataFrame, prefix: str, **meta) -> Path:
    """
    Save a DataFrame to a timestamped CSV file inside RAW folder.
    
    Args:
        df (pd.DataFrame): The dataframe to save.
        prefix (str): Prefix for the filename.
        **meta: Additional metadata (key=value) that will be embedded in the filename.
    
    Returns:
        Path: Full path of the saved CSV file.
    """
    # Clean metadata into filename-friendly string
    mid = "_".join([f"{k}-{v}" for k, v in meta.items() if v is not None])

    # Build filename and path
    filename = f"{prefix}_{mid}_{ts()}.csv"
    path = RAW / filename

    # Save CSV
    df.to_csv(path, index=False)

    print(f"✅ Saved CSV to: {path}")
    return path


# ---------------- Example Usage ----------------
# Assuming df_api already exists
_ = save_csv(
    df_api.sort_values('date'),
    prefix="api",
    source="alpha" if USE_ALPHA else "yfinance",
    symbol=SYMBOL
)

KeyError: 'date'

## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [13]:
def scrape_table(url: str, demo_html: str = None) -> pd.DataFrame:
    """
    Scrape the first <table> found at the given URL into a DataFrame.
    Falls back to demo_html if scraping fails.
    """
    try:
        resp = requests.get(url, headers=headers, timeout=30)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        print(f"✅ Successfully scraped: {url}")
    except Exception as e:
        print(f"⚠️ Scrape failed: {e}")
        if not demo_html:
            return pd.DataFrame()
        print("↩️ Using inline demo table as fallback.")
        soup = BeautifulSoup(demo_html, "html.parser")

    rows = [
        [c.get_text(strip=True) for c in tr.find_all(["th", "td"])]
        for tr in soup.find_all("tr")
    ]
    rows = [r for r in rows if r]

    if not rows:
        print("⚠️ No rows found in table.")
        return pd.DataFrame()

    header, *data = rows
    df = pd.DataFrame(data, columns=header)

    # Safe numeric conversion (no FutureWarning)
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except Exception:
            pass  # keep as string if not numeric

    return df


In [14]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

✅ Saved CSV to: raw\scrape_site-example_table-markets_20250820_102311.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.